---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [69]:
import pandas as pd
import numpy as np

def blight_model():
    
    # Your code here
    return get_ans()


def get_training_data ():
    df_train = pd.read_csv('train.csv', encoding="latin1")

    df_train1 = df_train[df_train.dtypes[(df_train.dtypes == "float64") | (df_train.dtypes=="int64")].index.values]

    # remove value with compliance NaN
    df_train1 = df_train1[~(np.isnan(df_train1['compliance']))]

    # skip columns with NaN values and which can cause data leakage
    skip_columns = [ 'ticket_id', 'payment_amount', 'balance_due', 'violation_zip_code', 
                 'mailing_address_str_number' ]
    df_train1 = df_train1[df_train1.columns.drop(skip_columns)]
    
    df_train1.fillna(value=0,inplace=True)

    return df_train1[df_train1.columns[:-1]], df_train1['compliance']


def get_test_data():
    df_test = pd.read_csv('test.csv', encoding="latin1")

    df_test1 = df_test[df_test.dtypes[(df_test.dtypes == "float64") | (df_test.dtypes=="int64")].index.values]

    ticket_ids = df_test1['ticket_id']
    
    # skip columns with NaN values and which can cause data leakage
    skip_columns = ['ticket_id', 'non_us_str_code' ]
    df_test1 = df_test1[df_test1.columns.drop(skip_columns)]

    return df_test1, ticket_ids

In [70]:
def get_ans():

    from sklearn.linear_model import LogisticRegression

    X_train, y_train = get_training_data()
    X_test, ticket_ids = get_test_data()

    y_scores_lr = LogisticRegression().fit(X_train, y_train).decision_function(X_test)

    return pd.Series(data=y_scores_lr, index=ticket_ids, dtype='float32')


In [72]:
# X_train, y_train = get_training_data()
# print(X_train.shape)
# print(X_train.dropna().shape)

# X_test, ticket_ids = get_test_data()
# print(X_test.shape)
# print(X_test.dropna().shape)
# print(X_train.columns)
# print(X_test.columns)

In [71]:
blight_model()

/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:1: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if __name__ == '__main__':


ticket_id
284932    -2.766591
285362    -5.092377
285361    -2.539205
285338    -2.712104
285346    -2.427522
285345    -2.711208
285347    -2.285679
285342    -1.442065
285530    -4.992035
284989    -3.604362
285344    -2.285898
285343    -4.980914
285340    -4.980502
285341    -2.285487
285349    -2.427183
285348    -2.710869
284991    -3.604190
285532    -3.571077
285406    -3.653046
285001    -3.562584
285006    -4.981013
285405    -5.095465
285337    -3.683563
285496    -2.274916
285497    -2.700445
285378    -5.098081
285589    -3.676471
285585    -2.813601
285501    -2.422117
285581    -5.100212
            ...    
376367    -4.314838
376366    -2.896409
376362    -2.967987
376363    -2.826144
376365    -4.314838
376364    -2.896409
376228    -2.858077
376265    -2.942158
376286    -1.377613
376320    -2.880547
376314    -2.936820
376327    -1.257119
376385    -1.293074
376435     0.493228
376370    -1.259622
376434    -2.398677
376459    -2.439879
376478   -16.340485
376473    